In [ ]:
import praw
import re
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# ✅ Initialize Reddit API client (Use your own credentials)
reddit = praw.Reddit(
    client_id="QWw_4O3xM-sXF-x42DjITQ",
    client_secret="3-kzXcwWZB9nXWc8YPIDjPX9rhOuJA",
    user_agent="reddit_comment_scraper by u/Agile-Bandicoot-6731"
)

# ✅ Initialize sentiment analyzer
# analyzer = SentimentIntensityAnalyzer()

# ✅ List of stop words (download NLTK stopwords if not already)
stop_words = set(stopwords.words('english'))

# ✅ Clean each comment
def clean_comment(comment):
    comment = comment.lower()  # Convert to lowercase
    comment = re.sub(r'http\S+', '', comment)  # Remove URLs
    comment = re.sub(r'u/[a-zA-Z0-9_]+', '', comment)  # Remove mentions
    comment = re.sub(r'[^a-zA-Z\s]', '', comment)  # Remove punctuation
    comment = re.sub(r'\s+', ' ', comment).strip()  # Remove extra spaces
    comment = ' '.join([word for word in comment.split() if word not in stop_words])  # Remove stopwords
    return comment

# ✅ Fetch comments from r/all
def fetch_comments_from_r_all(topic, post_limit=100):
    comments = []
    
    print(f"\n🔍 Searching in r/all for topic: {topic}")
    posts = reddit.subreddit('all').search(topic, limit=post_limit)
    
    for post in tqdm(posts, desc=f"Processing r/all posts"):
        try:
            post.comments.replace_more(limit=None)  # Fetch all comments
            for comment in post.comments.list():
                comments.append({
                    "comment": comment.body,
                    "subreddit": post.subreddit.display_name  # Store subreddit info
                })
        except Exception as e:
            print(f"Error fetching comments for a post: {e}")

    # ✅ Remove duplicate comments
    return comments

# ✅ Main execution
if __name__ == "__main__":
    topic = "AI in Education"
    print(f"\n🚀 Starting comment scraping for topic: {topic}")

    comments_data = fetch_comments_from_r_all(topic, post_limit=5)
    print(f"\n✅ Total unique comments fetched: {len(comments_data)}")
    
    # ✅ Convert to DataFrame for easy manipulation
    df = pd.DataFrame(comments_data)
    
    # ✅ Save to CSV file
    df.to_csv(f'comments_{topic.replace(" ", "_")}_all_subreddits.csv', index=False)
    print(f"\n💾 Comments saved to CSV file: comments_{topic.replace(' ', '_')}_all_subreddits.csv")
